# Workflow widgets example

This notebook illustrates how we can use ESSreduce’s [workflow widgets](https://scipp.github.io/essreduce/user-guide/widget.html) to generate a graphical interface for running the [DREAM GEANT4 workflow](./dream-powder-reduction.rst).

## Initializing the GUI

It is as simple as importing the dream submodule and generating a GUI using `workflow_widget` (the workflow automatically registers itself to a library of workflows when imported).

In [ ]:
# Import dream submodule to register workflow
from ess import dream
from ess.reduce import ui

# Prepare a container for accessing the results computed by the GUI
results = {}

# Initialize the GUI widget
widget = ui.workflow_widget(result_registry=results)
widget

In [ ]:
from ess.powder.types import DspacingBins, Filename, SampleRun, VanadiumRun
import ess.dream.data  # noqa: F401
from ess.dream import InstrumentConfiguration

select = widget.children[0].children[0]
keys, values = zip(*select.options, strict=True)
ind = keys.index("DreamGeant4ProtonChargeWorkflow")
select.value = values[ind]
# Select IofDspacing output
wfw = widget.children[1].children[0]
outputs = wfw.output_selection_box.typical_outputs_widget
keys, values = zip(*outputs.options, strict=True)
ind = keys.index("IofTof")
outputs.value = (values[ind],)
# Refresh parameters
pbox = wfw.parameter_box
pbox.parameter_refresh_button.click()
# Set parameters
pbox._input_widgets[Filename[SampleRun]].children[0].value = dream.data.simulated_diamond_sample()
pbox._input_widgets[Filename[VanadiumRun]].children[0].value = dream.data.simulated_vanadium_sample()
pbox._input_widgets[InstrumentConfiguration].value = InstrumentConfiguration.high_flux
pbox._input_widgets[DspacingBins].fields["stop"].value = 2.3434
# Run the workflow
rbox = wfw.result_box
rbox.run_button.click()

## Accessing the results

We can now access the computed result in the `results` dictionary:

In [ ]:
results

The result can be plotted using

In [ ]:
(da,) = results.values()
da.hist(tof=200).plot()